In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime as dt

%matplotlib inline

In [ ]:
full = pd.read_csv('../../data/raw/full_database.csv')

In [ ]:
recursos = pd.read_csv('../../data/raw/CSVs_Processados/recursos.csv')
recursos.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
execs = pd.read_csv('../../data/raw/CSVs_Processados/execucoes.csv', parse_dates=['Data_Execucao',
                                                                                   'Data_FechamentoCenario',
                                                                                   'TimeStamp_Execucao'])
execs.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
execs.shape

In [ ]:
execs = execs[pd.notnull(execs['Data_Execucao'])]

In [ ]:
#Filtrando testes executados a partir de 2018-12-01'
start_date = '2018-12-01'
execs = execs[execs['Data_Execucao'] >= start_date]

In [ ]:
execs.head()

---

Criando Base com informações **testers x testes totais**

In [ ]:
pivot_testes_por_tester = pd.pivot_table(execs, 
                                         values='ID_Execucao',
                                         columns=['Data_Execucao'],
                                         index = 'Testador_Real',
                                         aggfunc = 'count')

In [ ]:
pivot_testes_por_tester.shape

In [ ]:
pivot_testes_por_tester.head()

In [ ]:
pivot_testes_por_tester.reset_index(inplace = True)

In [ ]:
#criar coluna com contagem de dias trabalhados
pivot_testes_por_tester['contagem_testes_total'] = pivot_testes_por_tester.iloc[:,1:].count(axis=1)

In [ ]:
#criar coluna com contagem de testes totais
pivot_testes_por_tester['soma_testes_total'] = pivot_testes_por_tester.iloc[:,1:-1].sum(axis=1) 

In [ ]:
#criar coluna média de testes por dia
pivot_testes_por_tester['media_testes_total'] =  pivot_testes_por_tester['soma_testes_total']/pivot_testes_por_tester['contagem_testes_total']

In [ ]:
#Base com informações totais acerca do teste na base total
base_testers = pivot_testes_por_tester[['Testador_Real',
                                        'contagem_testes_total',
                                        'soma_testes_total',
                                        'media_testes_total']].copy()

In [ ]:
base_testers.rename({'Testador_Real':'testador_real'}, inplace = True, axis = 1)

In [ ]:
base_testers.head()

In [ ]:
#pivot apenas com testes billable
pivot_testes_por_tester_bill = pd.pivot_table(execs[execs['Target']==1], 
                                         values='ID_Execucao',
                                         columns=['Data_Execucao'],
                                         index = 'Testador_Real',
                                         aggfunc = 'count')
pivot_testes_por_tester_bill.reset_index(inplace = True)

In [ ]:
pivot_testes_por_tester_bill['soma_testes_total_bill'] = pivot_testes_por_tester_bill.iloc[:,1:].sum(axis=1) 

In [ ]:
adicionando colunas com número total de testes billable
base_testers = pd.merge(left=base_testers,
                        right=pivot_testes_por_tester_bill[['Testador_Real','soma_testes_total_bill']],
                        left_on='testador_real',
                        right_on='Testador_Real',
                        how='left')

In [ ]:
base_testers.drop('Testador_Real', axis=1, inplace = True)

In [ ]:
base_testers.head()

In [ ]:
base_testers['%_testes_billable'] = base_testers['soma_testes_total_bill']/base_testers['soma_testes_total']

In [ ]:
print('Média de testes billables: ' + str(base_testers['%_testes_billable'].mean()))
print('Desvpad de testes billables: ' + str(base_testers['%_testes_billable'].std()))
base_testers['%_testes_billable'].hist()

In [ ]:
base_testers.head()

---

Criando Base com informações testers x testes **por mês**

In [ ]:
execs['data_execuca_ano'] = pd.DatetimeIndex(execs['Data_Execucao']).year
execs['data_execuca_mes'] = pd.DatetimeIndex(execs['Data_Execucao']).month
execs['data_execuca_dia'] = pd.DatetimeIndex(execs['Data_Execucao']).day

In [ ]:
execs = execs.astype({'data_execuca_ano': str,
                      'data_execuca_mes': str,
                      'data_execuca_dia': str})
execs.info()

In [ ]:
execs['data_execuca_anomes'] = execs['data_execuca_ano'] + execs['data_execuca_mes']

In [ ]:
execs['data_execuca_anomesdia'] = execs['data_execuca_ano'] + execs['data_execuca_mes'] + execs['data_execuca_dia'] 

In [ ]:
#Testes por mês
pivot_testes_por_tester_mes = pd.pivot_table(execs, 
                                         values='ID_Execucao',
                                         columns=['data_execuca_anomes'],
                                         index='Testador_Real',
                                         aggfunc='count')
pivot_testes_por_tester_mes.reset_index(inplace = True)

In [ ]:
pivot_testes_por_tester_mes.head()

In [ ]:
#dias_trabalhados por mês por tester
pivot_testes_por_tester_mes_dias = pd.pivot_table(execs, 
                                         values='data_execuca_anomesdia',
                                         columns=['data_execuca_anomes'],
                                         index='Testador_Real',
                                         aggfunc=pd.Series.nunique)
pivot_testes_por_tester_mes_dias.reset_index(inplace = True)

In [ ]:
pivot_testes_por_tester_mes_dias.rename({'201812': '201812_dia',
                                    '20191': '20191_dia',
                                    '20192': '20192_dia',
                                    '20193': '20193_dia',
                                    '20194': '20194_dia'}, axis=1, inplace = True)
pivot_testes_por_tester_mes_dias.head()

In [ ]:
pivot_testes_por_tester_mes = pd.merge(pivot_testes_por_tester_mes,
                                       pivot_testes_por_tester_mes_dias,
                                       on='Testador_Real')

In [ ]:
pivot_testes_por_tester_mes['201812%'] = pivot_testes_por_tester_mes['201812']/pivot_testes_por_tester_mes['201812_dia']
pivot_testes_por_tester_mes['20191%'] = pivot_testes_por_tester_mes['20191']/pivot_testes_por_tester_mes['20191_dia']
pivot_testes_por_tester_mes['20192%'] = pivot_testes_por_tester_mes['20192']/pivot_testes_por_tester_mes['20192_dia']
pivot_testes_por_tester_mes['20193%'] = pivot_testes_por_tester_mes['20193']/pivot_testes_por_tester_mes['20193_dia']
pivot_testes_por_tester_mes['20194%'] = pivot_testes_por_tester_mes['20194']/pivot_testes_por_tester_mes['20194_dia']

In [ ]:
pivot_testes_por_tester_mes.head()

In [ ]:
#Testes por mês
pivot_testes_por_tester_mes_bill = pd.pivot_table(execs[execs['Target']==1], 
                                         values='ID_Execucao',
                                         columns=['data_execuca_anomes'],
                                         index='Testador_Real',
                                         aggfunc='count')
pivot_testes_por_tester_mes_bill.reset_index(inplace = True)

In [ ]:
pivot_testes_por_tester_mes_bill.rename({'201812': '201812_bill',
                                    '20191': '20191_bill',
                                    '20192': '20192_bill',
                                    '20193': '20193_bill',
                                    '20194': '20194_bill'}, axis=1, inplace = True)

In [ ]:
pivot_testes_por_tester_mes = pd.merge(pivot_testes_por_tester_mes,
                                       pivot_testes_por_tester_mes_bill,
                                       on='Testador_Real')

In [ ]:
pivot_testes_por_tester_mes.head()

In [ ]:
pivot_testes_por_tester_mes['201812%_bill'] =pivot_testes_por_tester_mes['201812_bill']/pivot_testes_por_tester_mes['201812']
pivot_testes_por_tester_mes['20191%_bill'] = pivot_testes_por_tester_mes['20191_bill']/pivot_testes_por_tester_mes['20191']
pivot_testes_por_tester_mes['20192%_bill'] = pivot_testes_por_tester_mes['20192_bill']/pivot_testes_por_tester_mes['20192']
pivot_testes_por_tester_mes['20193%_bill'] = pivot_testes_por_tester_mes['20193_bill']/pivot_testes_por_tester_mes['20193']
pivot_testes_por_tester_mes['20194%_bill'] = pivot_testes_por_tester_mes['20194_bill']/pivot_testes_por_tester_mes['20194']

In [ ]:
pivot_testes_por_tester_mes.head()

Dicionário de pivot_testes_por_tester_mes    
- **YYYYMM** = Testes realizados naquele mês por empregado    
- **YYYYMM_dia** = Dias trabalhados naquele mês por empregado   
- **YYYYMM%** = Média de testes realizados por mês por empregado   
- **YYYYMM_bill** = Testes faturados realizados por mês por empregado   
- **YYYYMM%_bill** = Testes faturados / Testes totais por mês por empregada

---

Checking Clusters

In [ ]:
# clusters = pd.read_csv('../../data/raw/CSVs_Processados/cluster.csv')

In [ ]:
# clusters.head()

In [ ]:
# base_testers = pd.merge(left=base_testers,
#                         right=clusters[['ALM USER','cluster']],
#                         left_on='testador_real',
#                         right_on='ALM USER',
#                         how='left')

---